# Gotland Basin

This notebook plots the vertical profiles of the simulated temperature at selected times when observations are available in the [Gotland deep](https://gotm.net/cases/gotland_deep/) test case.

In [ ]:
import sys
import numpy as np
import string
import matplotlib.pyplot as plt
# add the path of gotmtool
sys.path.append("../gotmtool")
from gotmtool import *

## Load data

First, store the information of the four simulations into lists and dictionaries.

In [ ]:
casenames = [
    'GLS-C01A',
    'KPP-CVMix',
    'KPPLT-VR12',
    'KPPLT-LF17',
    ]

colors = {
    'GLS-C01A': 'tab:blue',
    'KPP-CVMix': 'tab:olive',
    'KPPLT-VR12': 'tab:orange',
    'KPPLT-LF17': 'tab:red',
    }

casedir = '../gotm/run/Gotland_wave'
save_fig = True

Load data into a dictionary.

In [ ]:
sims = {}
dataset = {}
for case in casenames:
    sim = Simulation(path=casedir+'/'+case)
    sims[case] = sim
    dataset[case] = sim.load_data()

Get the time when observation is available.

In [ ]:
# get the time when observation is available
with open('gotland_obs_time.txt', 'r') as file:
    time_str = file.read().replace('/','-')
obs_time_str_list = time_str.split('\n')[:-1]
obs_time_list = [np.datetime64(ts) for ts in obs_time_str_list]

In [ ]:
data    = dataset['KPP-CVMix']
time    = data.time
time_start = time[0]
time_end = time[-1]
pfllist = []
for t in obs_time_list:
    if t > time_start and t < time_end:
        pfllist.append(t)

# Figure 1

Plot the vertical profiles of temperature and salinity when observations are available.

In [ ]:
data    = dataset['KPP-CVMix']
tpfl_obs =  data.temp_obs[:,:,0,0]
spfl_obs =  data.salt_obs[:,:,0,0]
for i, t in enumerate(pfllist):
    fig, axarr = plt.subplots(1,2, sharey='row')
    tpfl_obs.sel(time=t, method='nearest').plot(y='z', ax=axarr[0], color='k',
                  linewidth=1.5)
    spfl_obs.sel(time=t, method='nearest').plot(y='z', ax=axarr[1], color='k',
                  linewidth=1.5, label='Obs')
    for case in casenames:
        data = dataset[case]
        tpfl = dataset[case].temp[:,:,0,0].sel(time=t, method='nearest')
        tpfl.plot(y='z', ax=axarr[0], color=colors[case],
                  linewidth=1.5, alpha=0.8)
        spfl = dataset[case].salt[:,:,0,0].sel(time=t, method='nearest')
        spfl.plot(y='z', ax=axarr[1], color=colors[case],
                  linewidth=1.5, alpha=0.8, label=case)
    axarr[1].legend(loc='lower left')
    axarr[1].set_title('')
    axarr[1].set_ylabel('')
    if save_fig:
        os.makedirs('gotland_wave_pfls', exist_ok=True)
        figname = './gotland_wave_pfls/gotland_wave_pfl_{}.pdf'.format(
            str(pfllist[i]).replace(':',''))
        fig.savefig(figname, dpi=300)  
        plt.close(fig)
    

Show the time when observations are available.

In [ ]:
for i, t in enumerate(pfllist):
    print('{}: {}'.format(i, str(t)))

## Figure 2

A comparison of the temperature profiles at Gotland between the observation (black) and GOTM simulations (colored) at selected times.

In [ ]:
tidxlist1 = [10, 14, 18, 20]
tidxlist1_aux = [-1, 16, -1, 19]
tidxlist2 = [21, 27, 28, 29]
tidxlist2_aux = [-1, -1 , -1, -1]
fig, axarr = plt.subplots(2, 4, sharey='row')
fig.set_size_inches([8,6])

data    = dataset['KPP-CVMix']
tpfl_obs =  data.temp_obs[:,:,0,0]
spfl_obs =  data.salt_obs[:,:,0,0]
for i in np.arange(4):
    t = pfllist[tidxlist1[i]]
    tpfl_obs.sel(time=t, method='nearest').plot(y='z', ax=axarr[0,i], color='k',
                  linewidth=1.5)
    for case in casenames:
        data = dataset[case]
        tpfl = dataset[case].temp[:,:,0,0].sel(time=t, method='nearest')
        tpfl.plot(y='z', ax=axarr[0,i], color=colors[case],
                  linewidth=1.5, alpha=0.8)
    if tidxlist1_aux[i] > 0:
        t = pfllist[tidxlist1_aux[i]]
        tpfl_obs.sel(time=t, method='nearest').plot(y='z', ax=axarr[0,i], color='k',
                      linewidth=1.5, linestyle=':')
        for case in casenames:
            data = dataset[case]
            tpfl = dataset[case].temp[:,:,0,0].sel(time=t, method='nearest')
            tpfl.plot(y='z', ax=axarr[0,i], color=colors[case],
                      linewidth=1.5, linestyle=':', alpha=0.8)
        
    t = pfllist[tidxlist2[i]]
    tpfl_obs.sel(time=t, method='nearest').plot(y='z', ax=axarr[1,i], color='k',
                  linewidth=1.5, label='Obs')
    for case in casenames:
        data = dataset[case]
        tpfl = dataset[case].temp[:,:,0,0].sel(time=t, method='nearest')
        tpfl.plot(y='z', ax=axarr[1,i], color=colors[case],
                  linewidth=1.5, alpha=0.8, label=case)

    if tidxlist2_aux[i] > 0:
        t = pfllist[tidxlist2_aux[i]]
        tpfl_obs.sel(time=t, method='nearest').plot(y='z', ax=axarr[1,i], color='k',
                      linewidth=1.5, linestyle=':')
        for case in casenames:
            data = dataset[case]
            tpfl = dataset[case].temp[:,:,0,0].sel(time=t, method='nearest')
            tpfl.plot(y='z', ax=axarr[1,i], color=colors[case],
                      linewidth=1.5, linestyle=':', alpha=0.8)

axarr[1,0].legend(loc = 'lower left', fontsize=10, ncol=5,
                  bbox_to_anchor=(-0.04, -0.5), framealpha=1)
    
for i in np.arange(2):
    for j in np.arange(4):
        k = i*4+j
        ax = axarr[i,j]
        ax.set_title('')
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_ylim([-120,0])
        if j < 2:
            ax.text(0.05, 0.05, '('+string.ascii_lowercase[k]+')',
                    transform=ax.transAxes, fontsize=12, va='bottom', ha='left')
        else:
            ax.text(0.95, 0.05, '('+string.ascii_lowercase[k]+')',
                    transform=ax.transAxes, fontsize=12, va='bottom', ha='right')
    axarr[i,0].set_ylabel('Depth (m)')
    
for j in np.arange(4):
    axarr[0,j].set_title(str(pfllist[tidxlist1[j]])[:10])
    axarr[0,j].set_xlabel('$T$ ($^\circ$C)')
    axarr[1,j].set_title(str(pfllist[tidxlist2[j]])[:10])
    axarr[1,j].set_xlabel('$T$ ($^\circ$C)')
#     axarr[1,j].set_xlim([6.5,11])
    
if save_fig:
    plt.subplots_adjust(top=0.93, bottom=0.18, right=0.95, hspace=0.45, wspace=0.15)
    figname = 'gotland_wave_pfl.pdf'
    fig.savefig(figname, dpi=300)